In [1]:
import numpy as np

There are various types of dropout. Below, I have done the scratch implementation the inverted dropout. Inverted dropout has one simple twist in its implmentation that makes it more optimized than original standard dropout which has invented by ### hinton in 2012.

In future, I have a plan to implement other types of dropout like pooling-dropout, spatialdropout, etc which are used in CNN and other various types of dropout of RNN. 

In [2]:
X = np.array([[2],[3]])

In [3]:
X.shape

(2, 1)

In [4]:
Y = np.array([[1]])
Y.shape

(1, 1)

In [5]:
def parameters_initializer(l):
    '''
    initializes the parameters for each layer. 
    '''
    parameters = {}
    for i in range(1,len(l)):
        parameters["W"+str(i)] = np.random.randn(l[i],l[i-1])
        parameters["b"+str(i)] = np.zeros((l[i],1))
    return parameters

In [6]:
def linear_forward(X,W,b):
    #W.X + b
    z = np.dot(W,X) + b
        
    return z
        

In [7]:
def activation_pass(Z,activation,keepprob=1):
    if activation == 'relu':
        return np.where(0,Z<0,Z)
    else:
        return 1/(1+np.exp(-Z))
    

In [8]:
def activation_pass_dropout(Z,activation,keepprob):
    #implementation of inverted dropout
    if activation == 'relu':
        a = np.where(0,Z<0,Z)
        mask = np.random.rand(a.shape[0],a.shape[1]) < keepprob
        a = np.multiply(a,mask.astype(int))
        
        return a/keeprob
    
    else:
        a = 1/(1+np.exp(-Z))
        mask = np.random.rand(a.shape[0],a.shape[1]) < keepprob
        a = np.multiply(a,mask.astype(int))
        
        return a/keepprob
    

In [9]:
def cost_function(Y,yhat):
    '''
    binary cross-entropy as a cost function.
    '''
    m = Y.shape[1]
    return np.multiply(-1/m,np.dot(Y,yhat) + np.dot((1-Y),(1-yhat)))

In [10]:
def forward_prop(X,L,Y,keepprob=1):
    '''
    This is not full-fledged forward propagation. For now, all the layers will use sigmoid activation function.
    '''
    layer = L
    parameters = parameters_initializer(L)

    X = X
    caches = {}
    
    for i in range(1,len(layer)):
        z = linear_forward(X, parameters["W"+str(i)], parameters["b"+str(i)])
        
        if keepprob != 1: #if statement will run if the keepprob == 1 i.e., dropout is 0
            a = activation_pass_dropout(z,'sigmoid',keepprob)
            X = a
            
        else:
            a = activation_pass(z,'sigmoid')
            X = a

        caches['z' + str(i)] = z
        caches["a" + str(i)] = a
        
    cost_val = cost_function(Y,X)
    
    return caches, cost_val
    

In [11]:
L = [2,3,1]
forward_prop(X,L,Y,0.5)

({'z1': array([[3.76307577],
         [0.63885194],
         [4.26212776]]),
  'a1': array([[0.        ],
         [1.30898779],
         [1.97220709]]),
  'z2': array([[1.57045312]]),
  'a2': array([[0.]])},
 array([[-0.]]))